In [1]:
from stanfordcorenlp import StanfordCoreNLP
import json
from copy import deepcopy

In [ ]:
3760ff00e13513340363ec2e65cc9a925433d704

In [2]:
nlp = StanfordCoreNLP(r'stanford-corenlp-4.0.0') 

In [3]:
sentence = 'I like him.'
print('Part of Speech:', nlp.pos_tag(sentence))

Part of Speech: [('I', 'PRP'), ('like', 'VBP'), ('him', 'PRP'), ('.', '.')]


In [3]:
input_file = 'quoref-train-v0.1.json'

In [4]:
with open(input_file, "r", encoding='utf-8') as reader:
        input_data = json.load(reader)["data"]

In [5]:
def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [6]:
def remove_pro(paragraph_text):
    doc_tokens = []
    prev_is_whitespace = True
    for c in paragraph_text:
        if is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
    new_text=''
    for index,token in enumerate(doc_tokens):
        pos = nlp.pos_tag(token)
        origin_length = len(token)
        remove_pro = ''
        for item in pos:
            if item[1].find('PRP')==-1:
                remove_pro+=item[0]
        new_text += remove_pro+' '
    return new_text

In [12]:
new_data = deepcopy(input_data)

In [13]:
for entry in new_data:
    for paragraph in entry['paragraphs']:
        paragraph_text = paragraph["context"]
        #print(paragraph["context"])
        for qa in paragraph["qas"]:
            answer = qa["answers"][0]
            answer_offset = answer["answer_start"]
            orig_answer_text = answer["text"]
            new_answer_text = remove_pro(orig_answer_text)
            answer["text"] = new_answer_text[:-1]
            #print(paragraph_text[answer_offset:answer_offset+40])
            pre_text = paragraph_text[:answer_offset]
            new_text = remove_pro(pre_text)
            #print(len(new_text))
            answer["answer_start"] = len(new_text)
        new_text = remove_pro(paragraph_text)
        paragraph["context"]=new_text[:-1]

In [110]:
for entry in new_data:
    for paragraph in entry['paragraphs']:
        paragraph_text = paragraph["context"]
        if paragraph['context_id'] !='3760ff00e13513340363ec2e65cc9a925433d704':
            continue
        doc_tokens = []
        char_to_word_offset = []
        prev_is_whitespace = True
        for c in paragraph_text:
            if is_whitespace(c):
                prev_is_whitespace = True
            else:
                if prev_is_whitespace:
                    doc_tokens.append(c)
                else:
                    doc_tokens[-1] += c
                prev_is_whitespace = False
            char_to_word_offset.append(len(doc_tokens) - 1)
        #reduction, new_text = remove_pro(doc_tokens)
        #paragraph["context"] = new_text
        #print(paragraph_text)
        for qa in paragraph["qas"]:
            qas_id = qa["id"]
            question_text = qa["question"]
            start_position = None
            end_position = None
            orig_answer_text = None
            is_impossible = False
            if not is_impossible:
                answer = qa["answers"][0]
                orig_answer_text = answer["text"]
                answer_offset = answer["answer_start"]
                answer_length = len(orig_answer_text)
                print(answer_offset)
                start_position = char_to_word_offset[answer_offset]
                print(orig_answer_text,answer_length)
                end_position = char_to_word_offset[answer_offset + answer_length - 1]
                print(paragraph_text[answer_offset:answer_offset+answer_length])

377
cook 4
cook
358
Olga 4
Olga
231
playwright 10
playwright
811
engagement with another man 27
engagement with another man
1287
the producer of a successful musical show 41
the producer of a successful musical show


In [14]:
all_data={}
all_data['data']=new_data

In [15]:
input_file = 'quoref-train-v0.2.json'

In [16]:
with open(input_file, "w", encoding='utf-8') as reader:
        json.dump(all_data,reader,indent=2)